In [1]:
import pandas as pd

gap_train= pd.read_pickle('./temp_result/train_kaggle_processed_PPA_PCA_PPA')
gap_test= pd.read_pickle('./temp_result/test_kaggle_processed_PPA_PCA_PPA')
gap_valid= pd.read_pickle('./temp_result/valid_kaggle_processed_PPA_PCA_PPA')

In [2]:
import nltk
from nltk.tag import StanfordNERTagger
st = StanfordNERTagger('stanford-ner/english.all.3class.distsim.crf.ser.gz', 'stanford-ner/stanford-ner.jar')

def spans(txt):
    span = []
    tokens=nltk.word_tokenize(txt)
    offset = 0
    for token in tokens:
        offset = txt.find(token, offset)
        span.append((token, offset))
    return tokens, span

def extract_name(text):
    tokens, span = spans(text)
    name_span = []
    tags = st.tag(tokens)
    for ind, tag in enumerate(tags):
        if tag[1]=='PERSON': 
            name_span.append(span[ind])
    return name_span

#test['name_index'] = test['Text'].apply(lambda x: extract_name(x))

In [3]:
gap_train.head()

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,A-coref,B,B-offset,B-coref,...,B_dist,A_pos,B_pos,pron_pos,A_idx,B_idx,pron_idx,A_vector,B_vector,pron_vector
0,test-1,Upon their acceptance into the Kontinental Hoc...,His,383,Bob Suter,352,False,Dehner,366,True,...,0.008,0.444444,0.464052,0.490196,"[96, 97, 98, 99]","[101, 102, 103]",[109],"[[-0.16258414, 0.7170149, -0.16182709, 0.33984...","[[0.46089026, 0.8022368, -0.26556754, 0.248125...","[[0.6202348, -0.49657542, 0.5204739, -0.727411..."
1,test-2,"Between the years 1979-1981, River won four lo...",him,430,Alonso,353,True,Alfredo Di St*fano,390,False,...,0.014,0.253623,0.275362,0.300725,"[92, 93, 94]","[100, 101, 102, 103, 104, 105, 106, 107, 108]",[113],"[[-0.37001002, 0.51947534, 0.41240457, -0.0459...","[[0.66546816, 0.78631735, 0.57873654, 0.009659...","[[0.6709301, -0.5158502, 0.73226666, -0.564204..."
2,test-3,Though his emigration from the country has aff...,He,312,Ali Aladhadh,256,True,Saddam,295,False,...,0.008,0.408333,0.475000,0.508333,"[63, 64, 65, 66, 67, 68]","[75, 76]",[81],"[[-0.13203266, 1.3001504, 1.0933744, 0.0792686...","[[0.08869687, 0.7399757, -0.47687554, 0.293353...","[[0.37514523, -0.4730158, 0.10452151, -0.03368..."
3,test-4,"At the trial, Pisciotta said: ``Those who have...",his,526,Alliata,377,False,Pisciotta,536,True,...,-0.004,0.222951,0.321311,0.314754,"[108, 109, 110]","[143, 144, 145, 146]",[141],"[[-0.43812943, 0.23181425, -1.1097318, 0.02765...","[[-0.76229733, -0.70085824, -0.23845942, -0.59...","[[0.6802153, -0.041674256, 0.36125743, 0.46846..."
4,test-5,It is about a pair of United States Navy shore...,his,406,Eddie,421,True,Rock Reilly,559,False,...,-0.056,0.473118,0.607527,0.456989,"[113, 114]","[142, 143, 144, 145]",[110],"[[-0.3663241, 1.0233932, -0.040710405, -1.4244...","[[1.2342895, -0.16074917, -0.08441551, 0.13165...","[[0.31947732, -0.5680447, 1.1254325, -0.703441..."


In [4]:
gap_train['name_list'] = gap_train['Text'].apply(lambda x: extract_name(x))

In [5]:
gap_test['name_list'] = gap_test['Text'].apply(lambda x: extract_name(x))

In [6]:
gap_valid['name_list'] = gap_valid['Text'].apply(lambda x: extract_name(x))

In [7]:
def remove_AB(name_list, A, B):
    names = A.split(' ')+B.split(' ')
    names = [n.lower() for n in names]
    return [i for i in name_list if i[0].lower() not in names]

In [8]:
gap_train['name_list_2']  =  gap_train.apply(lambda x:remove_AB(x['name_list'],x["A"],x["B"]),axis = 1)
gap_valid['name_list_2']  =  gap_valid.apply(lambda x:remove_AB(x['name_list'],x["A"],x["B"]),axis = 1)
gap_test['name_list_2']  =  gap_test.apply(lambda x:remove_AB(x['name_list'],x["A"],x["B"]),axis = 1)

In [9]:
import numpy as np
def get_vector_index(name,offset,token_map):
    name = "".join(name.lower().split(" "))
    idx = 0
    s = ""
    res = []
    for i in sorted(token_map.keys()):
        idx = idx + 1
        if i < offset:
            continue
        else:
            s = s+token_map[i]
            res.append(idx)
            if s == name:
                break
    return np.array(res)    

In [10]:
def get_name_vector_index(name_list,token_map):
    idx = []
    for (n,o) in name_list:
        idx.extend(get_vector_index(n,o,token_map))
    return idx

In [11]:
gap_train['neither_idx'] = gap_train.apply(lambda x: get_name_vector_index(x["name_list"],x['token_map']), axis=1)
gap_test['neither_idx'] = gap_test.apply(lambda x: get_name_vector_index(x["name_list"],x['token_map']), axis=1)
gap_valid['neither_idx'] = gap_valid.apply(lambda x: get_name_vector_index(x["name_list"],x['token_map']), axis=1)

gap_train['neither_idx_2'] = gap_train.apply(lambda x: get_name_vector_index(x["name_list_2"],x['token_map']), axis=1)
gap_test['neither_idx_2'] = gap_test.apply(lambda x: get_name_vector_index(x["name_list_2"],x['token_map']), axis=1)
gap_valid['neither_idx_2'] = gap_valid.apply(lambda x: get_name_vector_index(x["name_list_2"],x['token_map']), axis=1)

In [12]:
def del_idx(idx_1,idx_2):
    return sorted(set(idx_1)-set(idx_2))

In [13]:
gap_train['neither_idx'] = gap_train.apply(lambda x: del_idx(x["neither_idx"],x['pron_idx']), axis=1)
gap_test['neither_idx'] = gap_test.apply(lambda x: del_idx(x["neither_idx"],x['pron_idx']), axis=1)
gap_valid['neither_idx'] = gap_valid.apply(lambda x: del_idx(x["neither_idx"],x['pron_idx']), axis=1)

gap_train['neither_idx_2'] = gap_train.apply(lambda x: del_idx(x["neither_idx_2"],x['pron_idx']), axis=1)
gap_test['neither_idx_2'] = gap_test.apply(lambda x: del_idx(x["neither_idx_2"],x['pron_idx']), axis=1)
gap_valid['neither_idx_2'] = gap_valid.apply(lambda x: del_idx(x["neither_idx_2"],x['pron_idx']), axis=1)

In [14]:
gap_train['neither_idx'] = gap_train.apply(lambda x: del_idx(x["neither_idx"],x['A_idx']), axis=1)
gap_test['neither_idx'] = gap_test.apply(lambda x: del_idx(x["neither_idx"],x['A_idx']), axis=1)
gap_valid['neither_idx'] = gap_valid.apply(lambda x: del_idx(x["neither_idx"],x['A_idx']), axis=1)

gap_train['neither_idx_2'] = gap_train.apply(lambda x: del_idx(x["neither_idx_2"],x['A_idx']), axis=1)
gap_test['neither_idx_2'] = gap_test.apply(lambda x: del_idx(x["neither_idx_2"],x['A_idx']), axis=1)
gap_valid['neither_idx_2'] = gap_valid.apply(lambda x: del_idx(x["neither_idx_2"],x['A_idx']), axis=1)

In [15]:
gap_train['neither_idx'] = gap_train.apply(lambda x: del_idx(x["neither_idx"],x['B_idx']), axis=1)
gap_test['neither_idx'] = gap_test.apply(lambda x: del_idx(x["neither_idx"],x['B_idx']), axis=1)
gap_valid['neither_idx'] = gap_valid.apply(lambda x: del_idx(x["neither_idx"],x['B_idx']), axis=1)

gap_train['neither_idx_2'] = gap_train.apply(lambda x: del_idx(x["neither_idx_2"],x['B_idx']), axis=1)
gap_test['neither_idx_2'] = gap_test.apply(lambda x: del_idx(x["neither_idx_2"],x['B_idx']), axis=1)
gap_valid['neither_idx_2'] = gap_valid.apply(lambda x: del_idx(x["neither_idx_2"],x['B_idx']), axis=1)

In [16]:
gap_train['neither_vector'] = gap_train.apply(lambda x: x["vector"][x['neither_idx'],:], axis=1)
gap_test['neither_vector'] = gap_test.apply(lambda x: x["vector"][x['neither_idx'],:], axis=1)
gap_valid['neither_vector'] = gap_valid.apply(lambda x: x["vector"][x['neither_idx'],:], axis=1)

gap_train['neither_vector_2'] = gap_train.apply(lambda x: x["vector"][x['neither_idx_2'],:], axis=1)
gap_test['neither_vector_2'] = gap_test.apply(lambda x: x["vector"][x['neither_idx_2'],:], axis=1)
gap_valid['neither_vector_2'] = gap_valid.apply(lambda x: x["vector"][x['neither_idx_2'],:], axis=1)

In [17]:
gap_train.to_pickle('./temp_result/train_kaggle_processed_PPA_PCA_PPA_neither')
gap_test.to_pickle('./temp_result/test_kaggle_processed_PPA_PCA_PPA_neither')
gap_valid.to_pickle('./temp_result/valid_kaggle_processed_PPA_PCA_PPA_neither')